In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd

from game import Game
from player import Player

In [2]:
from your_name import YourName

In [3]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [4]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(9, 0), (12, 1), (14, 1), (10, 1), (14, 0), (14, 3), (10, 0), (14, 2)]
[(11, 1), (9, 3), (11, 0), (11, 3), (9, 2), (13, 3), (12, 2), (9, 1)]


In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [6]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 1000
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                #print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [7]:
strategy = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ]

In [8]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results
        


==== 0 RandomPlayer
1 YourName
==== 1 YourName


In [9]:
full_results

[[None,
  [[1000, 0],
   [0, 0],
   [8.0, 7.529],
   [0.0, 15.0],
   1.0,
   [0.0, 0.0],
   [0.0, 7.529],
   [2.595, 0.0],
   [0, 0],
   0,
   0]],
 [None, None]]

Simple stats

In [10]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(A)
        S = str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
    
for a in ANALYZE_STATS:
    printMatrix(full_results, a)



Wins
 RANDOM YourName 
0
RANDOM - 1000 
1
YourName 0 - 
Draws
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 
Moves
 RANDOM YourName 
0
RANDOM - 8.0 
1
YourName 7.529 - 
Cards
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 15.0 - 
Checks
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 0.0 - 
Draw Decisions
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 7.529 - 
Cheats
 RANDOM YourName 
0
RANDOM - 2.595 
1
YourName 0.0 - 
Errors
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 


### __________________________________________My Code ______________________________________________

In [11]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ["RandomPlayer2", "RANDOM", RandomPlayer]]

def compare_all(agent_strategies, verbose = 1):
    wins = {agent_strategies[i][0]: {agent_strategies[i][0]: None} for i in range(len(agent_strategies))}
    for i in range(len(agent_strategies)):
        for ii in range(i+1, len(agent_strategies)):
            player1 = agent_strategies[i]
            player2 = agent_strategies[ii]
            strategy = [player1, player2]
            for A in range(len(strategy)):
                for B in range(A+1,len(strategy)):
                    results = comparePlayers(strategy[A][2], strategy[B][2])
                    full_results[A][B] = results
                    wins[strategy[A][0]][strategy[B][0]] = full_results[0][1][0][1]
                    wins[strategy[B][0]][strategy[A][0]] = full_results[0][1][0][0]
                    if verbose:
                        if full_results[0][1][8][0] > 0 or full_results[0][1][8][1] > 0 or full_results[0][1][9] > 0:
                            print(f"Errors occured in game: {strategy[A][0]} vs {strategy[B][0]}")
    df = pd.DataFrame(wins)
    df['|| Total Wins ||'] = df.fillna(0).sum(axis=1)
    df = df.sort_values(by='|| Total Wins ||', ascending=False)

    dominated_players = []
    temp_df = df.drop(columns=["|| Total Wins ||"])
    for i, row in temp_df.iterrows():
        num_dominated = 0
        for column in temp_df.columns:
            if i != column and temp_df[column][i] > temp_df[i][column]:
                num_dominated += 1
        dominated_players.append(num_dominated)
    df['|| Dominated Players ||'] = dominated_players
    return df

compare_all(agent_strategies)

,RandomPlayer,YourName,RandomPlayer2,|| Total Wins ||,|| Dominated Players ||
RandomPlayer,NaN,1000.0,501.0,1501.0,2
RandomPlayer2,499.0,1000.0,NaN,1499.0,1
YourName,0.0,NaN,0.0,0.0,0


In [12]:
from Test_Players import *
from Adam_player import *
from B import *

In [13]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["Saint_Sceptic_25_Naive", "SAINT", Saint_Sceptic_25_Naive],
            ["Saint_Sceptic_50_Naive", "SAINT", Saint_Sceptic_50_Naive],
            ["Saint_Sceptic_75_Naive", "SAINT", Saint_Sceptic_75_Naive],
            ["Saint_Sceptic_Adaptable_Naive", "SAINT", Saint_Sceptic_Adaptable_Naive],
            ["YourName", "YourName", YourName],
           ["MyRandom_Default", "CUSTOM_RANDOM", MyRandom_Default],
           ["MyRandom_All_Cards", "CUSTOM_RANDOM", MyRandom_All_Cards],
           ["MyRandom_Real_Cards", "CUSTOM_RANDOM", MyRandom_Real_Cards],
           ["Random_Never_Accuse", "CUSTOM_RANDOM", Random_Never_Accuse],
           ["Random_Always_Accuse", "CUSTOM_RANDOM", Random_Always_Accuse],
           ["Mimic", "CUSTOM_RANDOM", Mimic],
           ["Saint_Believer", "SAINT", Saint_Believer],
           ["Saint_Once_in_a_While", "SAINT", Saint_Once_in_a_While],
           ["Saint_Two_Face", "SAINT", Saint_Two_Face],
           ["Saint_Accountant", "SAINT", Saint_Accountant],
           ["Mat_player", "SAINT", Saint_Nervous],
           ["Mat_player_ver_2", "SAINT", Saint_Nervous_ver_2],
           ["Mat_Player_ver_3", "SAINT", Saint_Nervous_ver_3],
           ["Adam_player", "CUSTOM", Adam_player],
           ["Saint_Nervous_Memory", "SAINT", Saint_Nervous_Memory],
           ["Saint_Collector", "SAINT", Saint_Collector],
           ["Dabbler_Believer", "DABBLER", Dabbler_Believer],
           ["Dabbler_Once_in_a_While", "DABBLER", Dabbler_Once_in_a_While],
           ["Dabbler_Two_Face", "DABBLER", Dabbler_Two_Face],
           ["Dabbler_Accountant", "DABBLER", Dabbler_Accountant],
           ["Dabbler_Nervous", "DABBLER", Dabbler_Nervous],
           ["Dabbler_Collector", "DABBLER", Dabbler_Collector],
           ["Coin_Flipper_Believer", "COIN_FLIPPER", Coin_Flipper_Believer],
           ["Coin_Flipper_Once_in_a_While", "COIN_FLIPPER", Coin_Flipper_Once_in_a_While],
           ["Coin_Flipper_Two_Face", "COIN_FLIPPER", Coin_Flipper_Two_Face],
           ["Coin_Flipper_Accountant", "COIN_FLIPPER", Coin_Flipper_Accountant],
           ["Coin_Flipper_Nervous", "COIN_FLIPPER", Coin_Flipper_Nervous],
           ["Coin_Flipper_Collector", "COIN_FLIPPER", Coin_Flipper_Collector],
           ["Pathological_Believer", "PATHOLOGICAL", Pathological_Believer],
           ["Pathological_Once_in_a_While", "PATHOLOGICAL", Pathological_Once_in_a_While],
           ["Pathological_Two_Face", "PATHOLOGICAL", Pathological_Two_Face],
           ["Pathological_Accountant", "PATHOLOGICAL", Pathological_Accountant],
           ["Pathological_Nervous", "PUSHER", Pathological_Nervous],
           ["Pathological_Collector", "PUSHER", Pathological_Collector],
           ["Pusher_Believer", "PUSHER", Pusher_Believer],
           ["Pusher_Once_in_a_While", "PUSHER", Pusher_Once_in_a_While],
           ["Pusher_Two_Face", "PUSHER", Pusher_Two_Face],
           ["Pusher_Accountant", "PUSHER", Pusher_Accountant],
           ["Pusher_Nervous", "PUSHER", Pusher_Nervous],
           ["Pusher_Collector", "PUSHER", Pusher_Collector],
           ["B_1", "B_PLAYER", Blayer],
           ["B_2", "B_PLAYER", Blayer2],
           ["B_3", "B_PLAYER", Blayer3],
           ["Saint_Nervous_ver_B", "B_PLAYER", Saint_Nervous_ver_B],
           ["Saint_Nervous_ver_B2", "B_PLAYER", Saint_Nervous_ver_B2],
           ["Saint_Nervous_ver_B3", "B_PLAYER", Saint_Nervous_ver_B3],
           ["Saint_Nervous_ver_B4", "B_PLAYER", Saint_Nervous_ver_B4],
           ]

results = compare_all(agent_strategies)
results

,RandomPlayer,Saint_Sceptic_25_Naive,Saint_Sceptic_50_Naive,Saint_Sceptic_75_Naive,Saint_Sceptic_Adaptable_Naive,YourName,MyRandom_Default,MyRandom_All_Cards,MyRandom_Real_Cards,Random_Never_Accuse,...,Pusher_Collector,B_1,B_2,B_3,Saint_Nervous_ver_B,Saint_Nervous_ver_B2,Saint_Nervous_ver_B3,Saint_Nervous_ver_B4,|| Total Wins ||,|| Dominated Players ||
Mat_Player_ver_3,1000.0,1000.0,1000.0,1000.0,833.0,1000.0,1000.0,1000.0,999.0,993.0,...,933.0,965.0,1000.0,992.0,930.0,952.0,651.0,687.0,42776.0,47
Mat_player_ver_2,1000.0,1000.0,1000.0,1000.0,827.0,1000.0,1000.0,1000.0,999.0,994.0,...,941.0,975.0,1000.0,994.0,925.0,956.0,658.0,637.0,42749.0,45
Mat_player,1000.0,1000.0,1000.0,1000.0,783.0,1000.0,1000.0,1000.0,1000.0,981.0,...,927.0,964.0,1000.0,990.0,908.0,944.0,659.0,658.0,41855.0,46
Saint_Nervous_Memory,999.0,931.0,929.0,906.0,590.0,1000.0,1000.0,1000.0,1000.0,989.0,...,942.0,964.0,1000.0,995.0,930.0,943.0,368.0,321.0,39582.0,41
Dabbler_Nervous,651.0,857.0,872.0,918.0,861.0,1000.0,749.0,771.0,758.0,955.0,...,821.0,761.0,645.0,686.0,765.0,788.0,714.0,709.0,35076.0,43
Saint_Accountant,949.0,1000.0,1000.0,1000.0,337.0,1000.0,997.0,999.0,996.0,981.0,...,912.0,909.0,951.0,914.0,923.0,935.0,196.0,191.0,34719.0,36
Saint_Two_Face,995.0,1000.0,1000.0,1000.0,115.0,1000.0,1000.0,1000.0,996.0,975.0,...,878.0,932.0,990.0,965.0,873.0,865.0,128.0,115.0,33269.0,35
Saint_Sceptic_Adaptable_Naive,731.0,1000.0,1000.0,1000.0,NaN,1000.0,955.0,987.0,929.0,935.0,...,630.0,713.0,739.0,706.0,665.0,641.0,388.0,351.0,32498.0,35
Dabbler_Accountant,608.0,564.0,665.0,785.0,590.0,1000.0,987.0,993.0,981.0,984.0,...,913.0,908.0,817.0,894.0,914.0,916.0,432.0,447.0,32090.0,35
Dabbler_Two_Face,886.0,602.0,625.0,687.0,361.0,1000.0,992.0,996.0,968.0,972.0,...,925.0,902.0,849.0,881.0,902.0,816.0,80.0,120.0,31571.0,33


In [14]:
print("B1 vs Saint Nervous v2")
comparePlayers(Blayer, Saint_Nervous_ver_2)

B1 vs Saint Nervous v2


[[333, 667],
 [0, 0],
 [9.422, 9.6],
 [3.605, 1.634],
 10.761,
 [0.0, 1.078],
 [1.258, 0.0],
 [0.0, 0.0],
 [0, 0],
 0,
 0]

In [15]:
print("Saint_Nervous_ver_2 vs B2")
comparePlayers(Saint_Nervous_ver_2, Blayer2)

Saint_Nervous_ver_2 vs B2


[[667, 333],
 [0, 0],
 [9.437, 9.262],
 [1.615, 3.446],
 10.939,
 [1.018, 0.0],
 [0.0, 1.177],
 [0.0, 0.0],
 [0, 0],
 0,
 0]

In [16]:
print("B1 vs B2")
comparePlayers(Blayer, Blayer2)

B1 vs B2


[[487, 513],
 [0, 0],
 [8.903, 8.9],
 [2.545, 2.435],
 11.02,
 [0.0, 0.0],
 [0.862, 0.835],
 [0.0, 0.0],
 [0, 0],
 0,
 0]

In [17]:
print("B vs Adam")
comparePlayers(Blayer, Adam_player)

B vs Adam


[[996, 4],
 [100, 100],
 [18.142, 17.715],
 [0.471, 11.741],
 3.788,
 [0.083, 9.11],
 [4.815, 0.003],
 [0.0, 0.185],
 [0, 0],
 0,
 100]

In [18]:
print("B2 vs Adam")
comparePlayers(Blayer2, Adam_player)

B2 vs Adam


[[995, 5],
 [98, 98],
 [17.958, 17.541],
 [0.432, 11.701],
 3.867,
 [0.077, 9.01],
 [4.71, 0.0],
 [0.0, 0.199],
 [0, 0],
 0,
 98]

In [19]:
print("Mat vs Adam")
comparePlayers(Saint_Nervous, Adam_player)

Mat vs Adam


[[998, 2],
 [0, 0],
 [8.748, 8.269],
 [0.005, 11.634],
 4.361,
 [0.312, 4.513],
 [0.0, 0.001],
 [0.0, 0.188],
 [0, 0],
 0,
 0]

In [20]:
print("Mat2 vs Adam")
comparePlayers(Saint_Nervous_ver_2, Adam_player)

Mat2 vs Adam


[[998, 2],
 [0, 0],
 [8.721, 8.252],
 [0.008, 11.671],
 4.321,
 [0.331, 4.46],
 [0.0, 0.001],
 [0.0, 0.165],
 [0, 0],
 0,
 0]

In [23]:
print("Mat3 vs Adam")
comparePlayers(Saint_Nervous_ver_3, Adam_player)

Mat3 vs Adam


[[1000, 0],
 [0, 0],
 [8.856, 8.36],
 [0.0, 11.815],
 4.185,
 [0.365, 4.586],
 [0.0, 0.003],
 [0.0, 0.133],
 [0, 0],
 0,
 0]

In [24]:
print("Random vs Adam")
comparePlayers(RandomPlayer, Adam_player)

Random vs Adam


[[1, 999],
 [0, 0],
 [14.028, 14.53],
 [15.167, 0.014],
 0.819,
 [7.307, 9.769],
 [0.0, 0.046],
 [7.999, 0.294],
 [0, 0],
 0,
 0]